In [3]:
from pydantic import BaseModel, Field

class Memory(BaseModel):
    content: str = Field(description="The main content of the memory")

class MemoryCol(BaseModel):
    memories: list[Memory] = Field(description="A list of memories about hte user.")



In [5]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
 
# Blind schema at model
model_with_structure = model.with_structured_output(MemoryCol)

# Invoke the model to produce strutured output that matches the schema
memory_collection = model_with_structure.invoke([HumanMessage("My name is Lance. I like to bike")])
memory_collection.memories

[Memory(content="User's name is Lance."),
 Memory(content='User likes to bike.')]

In [7]:
import uuid
from langgraph.store.memory import InMemoryStore

# Initialize the in-memory store
in_memory_store=InMemoryStore()

# Namespace for the memory to save
user_id="1"
namespace=(user_id, "memories")

# Save a memory to namespace as key and value
key = str(uuid.uuid4())
value = memory_collection.memories[0].model_dump()
in_memory_store.put(namespace, key, value)

key = str(uuid.uuid4())
value = memory_collection.memories[1].model_dump()
in_memory_store.put(namespace, key, value)


In [8]:
for m in in_memory_store.search(namespace):
    print(m.dict())

{'namespace': ['1', 'memories'], 'key': '0625131f-9e04-4474-b109-bf8800c84bef', 'value': {'content': "User's name is Lance."}, 'created_at': '2025-03-07T13:41:57.993288+00:00', 'updated_at': '2025-03-07T13:41:57.993291+00:00', 'score': None}
{'namespace': ['1', 'memories'], 'key': '996e5955-c50e-46c0-8c71-81c097331ef8', 'value': {'content': 'User likes to bike.'}, 'created_at': '2025-03-07T13:41:57.993449+00:00', 'updated_at': '2025-03-07T13:41:57.993450+00:00', 'score': None}


In [9]:
from trustcall import create_extractor

# Create the extractor
trustcall_extractor = create_extractor(
    model,
    tools = [Memory],
    tool_choice="Memory",
    enable_inserts = True,
)

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Instruction
instructions = "Extract memories"

 # Conversation 
conversation = [HumanMessage(content="Hi, I'm Narcy."), 
                AIMessage(content="Nice to meet you, Narcy."), 
                HumanMessage(content="This morning I had a nice bike ride in Hyderabad.")]

# Invoke the extractor
result  = trustcall_extractor.invoke({"messages":[SystemMessage(content=instructions)]+ conversation})


In [11]:
for m in result['messages']:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_FtRw96I3zr9U2mb4wNvq77Q2)
 Call ID: call_FtRw96I3zr9U2mb4wNvq77Q2
  Args:
    content: Narcy had a nice bike ride in Hyderabad this morning.


In [16]:
# Update the conversation
updated_conversation = [AIMessage(content="That's great, did you do after?"), 
                        HumanMessage(content="I went to Tartine and ate a croissant."),                        
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my Japan, and going back this winter!"),]

system_message = "Update existing memories and create new ones based on the following conversation"

tool_name = "Memory"
existing_memories = [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result["responses"])] if result['responses'] else None
existing_memories

[('0',
  'Memory',
  {'content': 'Narcy had a nice bike ride in Hyderabad this morning. Later, I went to Tartine and ate a croissant. I was thinking about my Japan, and going back this winter!'}),
 ('1',
  'Memory',
  {'content': 'Narcy had a nice bike ride in Hyderabad this morning. Later, I went to Tartine and ate a croissant. I was thinking about my Japan, and going back this winter!'})]

In [17]:
# Invoke the extractor with updated convo
result = trustcall_extractor.invoke({"messages": updated_conversation, 
                                     "existing":existing_memories})

In [18]:
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_fWDoObj2Hx675h2gYXu10QAG)
 Call ID: call_fWDoObj2Hx675h2gYXu10QAG
  Args:
    content: I went to Tartine and ate a croissant.
  Memory (call_Ev1LkAZuGTEIVvu3HvMuy0DV)
 Call ID: call_Ev1LkAZuGTEIVvu3HvMuy0DV
  Args:
    content: I was thinking about my Japan, and going back this winter!


In [19]:
# Response contains the memories
for m in result["responses"]:
    print(m)

content='I went to Tartine and ate a croissant.'
content='I was thinking about my Japan, and going back this winter!'
